In [1]:
import torch
import nestedtensor
from IPython.display import Markdown, display
def print_eval(s):
    colorS = "<span style='color:darkred'>$ {}</span>".format(s)
    display(Markdown('**{}**'.format(colorS))) 
    print('{}\n'.format(str(eval(s))))

In [2]:
import time as time_module
def time(fn):
    t0 = time_module.time()
    count = 0
    past = 0
    while past < 10.0:
        fn()
        past = time_module.time() - t0
        count += 1
    past = past / count
    return "average {:2.4f}ms based on {} samples".format(past * 1000, count)

In [6]:
def generate_tensors(num_tensor, vocab_size):
    sentence_lengths = torch.normal(75.0, 10.0, size=(num_tensor,)).long()
    return [(torch.rand(l) * vocab_size).long() for l in sentence_lengths]

class TextSentiment(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim, sparse=True)
        self.fc = torch.nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text):
        emb = self.embedding(text)
        return self.fc(emb)

vocab_size = 10000
model = TextSentiment(10000, 256, 5)
nt_text = nestedtensor.nested_tensor(generate_tensors(4, 10000), dtype=torch.int64)
print_eval("model(nt_text).nested_size()")

**<span style='color:darkred'>$ model(nt_text).nested_size()</span>**

NestedSize([
	torch.Size([76, 5]),
	torch.Size([74, 5]),
	torch.Size([74, 5]),
	torch.Size([76, 5])
])

